In [54]:
import tensorflow as tf
import SimpleITK as sitk
import numpy as np
import csv
import os

In [2]:
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[ 22.  28.]
 [ 49.  64.]]


In [64]:
print(image)

Image (0x3aebad0)
  RTTI typeinfo:   itk::Image<int, 3u>
  Reference Count: 1
  Modified Time: 323742
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 303841
  UpdateMTime: 323741
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 601]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 601]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 601]
  Spacing: [0.621094, 0.621094, 0.5]
  Origin: [-170.1, -135.2, -318.75]
  Direction: 
1 0 0
0 1 0
0 0 1

  IndexToPointMatrix: 
0.621094 0 0
0 0.621094 0
0 0 0.5

  PointToIndexMatrix: 
1.61006 0 0
0 1.61006 0
0 0 2

  Inverse Direction: 
1 0 0
0 1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (0x3aebdb0)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, int>
      Reference

In [105]:
print(all(range(2,5)))


True


In [80]:
sitk.Resample?

In [108]:
labels_path   = "./Data/labels.csv" 
series_path   = "./Data/series/"
np_path       = "./Data/np/"
shrink_factors= range(2, 4)
ref_dim       = (602, 512, 512)


for shrink_factor in shrink_factors:

    entries = []
    scale_factor  = 1/shrink_factor
    out_dim = (int(ref_dim[0] * scale_factor), int(ref_dim[1] * scale_factor), int(ref_dim[2] * scale_factor))
    
    with open(labels_path, 'r') as csvfile:
        csv_reader = csv.DictReader(csvfile, delimiter=',', quotechar='|')
        for row in csv_reader:
            record_id = row['record_id']
            if row['emphysema'] != '':
                emphysema = int(row['emphysema'])
            else:
                emphysema = 0

            if row['bronchiectasis1'] != '':
                bronchiectasis = int(row['bronchiectasis1'])
            else:
                bronchiectasis = 0
            
            print("Processing subject {0} at shrink factor {1}".format(record_id, shrink_factor))
            subject_path = series_path + record_id
            if not os.path.isdir(subject_path):
                print("Missing subject {0}".format(subject_path))
                continue

            reader = sitk.ImageSeriesReader()
            reader.SetFileNames(reader.GetGDCMSeriesFileNames(subject_path))
            image = reader.Execute()

            image = sitk.Shrink(image, [shrink_factor, shrink_factor, shrink_factor])

            arr = pad_image(image, out_dim)
            entries.append([arr, emphysema])
        arr = np.array(entries)
        print("Array size = {0} bytes".format(getsizeof(arr)))
        np.savez(np_path + 'data_' + str(shrink_factor)  , arr)
        


Processing subject CU100010 at shrink factor 2
Processing subject CU100023 at shrink factor 2


KeyboardInterrupt: 

In [62]:
def pad_image(image, target_dim):
    arr = sitk.GetArrayFromImage(image)
    
    padding_map = []
    for i in range(len(target_dim)):
        padding_map.append( [0, max(0, target_dim[i] - arr.shape[i])] )
        
    x = np.pad(arr, padding_map, mode='constant',constant_values=0)
    return np.int16(x)


In [48]:
myFilter = BinaryThresholdImageFilter()
myFilter.SetDebug(False)
myFilter.SetInsideValue(1)
myFilter.SetLowerThreshold(-1000.0)
myFilter.SetNumberOfThreads(8)
myFilter.SetOutsideValue(0)
myFilter.SetUpperThreshold(-400.0)

seed_left  = (116, 275, 300)
seed_right = (261, 2  , 300)
myFilter = NeighborhoodConnectedImageFilter()
myFilter.SetDebug(False)
myFilter.SetLower(1.0)
myFilter.SetNumberOfThreads(8)
myFilter.SetRadius((3, 3, 3))
myFilter.SetReplaceValue(1.0)
myFilter.SetSeedList((seed_left, seed_right))
myFilter.SetUpper(1.0)


myFilter = BinaryDilateImageFilter()
myFilter.SetBackgroundValue(0.0)
myFilter.SetBoundaryToForeground(False)
myFilter.SetDebug(False)
myFilter.SetForegroundValue(1.0)
myFilter.SetKernelRadius((5, 5, 5))
myFilter.SetKernelType(1)
myFilter.SetNumberOfThreads(8)

myFilter = BinaryErodeImageFilter()
myFilter.SetBackgroundValue(0.0)
myFilter.SetBoundaryToForeground(True)
myFilter.SetDebug(False)
myFilter.SetForegroundValue(1.0)
myFilter.SetKernelRadius((5, 5, 5))
myFilter.SetKernelType(1)
myFilter.SetNumberOfThreads(8)


-3024
